In [ ]:
#Project pipeline
# Text cleaning ----- ??
# Coreference resolution  ------ To be done?
# Named entity recognition
# Entity linking
# Relation extraction

In [2]:
import matplotlib.pyplot as plt
from collections import Counter
from nltk.corpus import reuters
import re
import spacy
nlp = spacy.load('en_core_web_lg') #, disable=['parser', 'tagger'])
reuters_fileids_crude = reuters.fileids(categories=['crude'])
# reuters_nlp = [nlp(re.sub('\s+',' ', reuters.raw(i).strip())) for i in reuters_fileids[:100]]
reuters_nlp_crude = [nlp(re.sub('\s+',' ', reuters.raw(i).strip())) for i in reuters_fileids_crude]
reuters_nlp_crude
length = [[i,  reuters.raw(i).strip()] for i in reuters_fileids_crude]
# label_counter = Counter()
# reuters.words()
# reuters.sents()
# doc = reuters_nlp[50]

In [3]:
doc = reuters_nlp_crude
len(doc)
# doc
# [len(reuters_nlp[i]) for i in reuters_fileids(categories=['fuel'])] #reuters_fileids(categories=['fuel']

# len(reuters.paras())
# reuters.paras()[5]

578

In [38]:
#attemp in putting everything into a dataframe
import pandas as pd
data = pd.DataFrame(reuters.fileids('crude'),columns=['fileids'])
data[]

In [49]:
title = []
for i in range(2):
    for j in doc[i]:
        s = str(j)
        if s.isupper():
            title = str(j).lower()

            

title

'u.s.'

In [50]:
doc[0]

JAPAN TO REVISE LONG-TERM ENERGY DEMAND DOWNWARDS The Ministry of International Trade and Industry (MITI) will revise its long-term energy supply/demand outlook by August to meet a forecast downtrend in Japanese energy demand, ministry officials said. MITI is expected to lower the projection for primary energy supplies in the year 2000 to 550 mln kilolitres (kl) from 600 mln, they said. The decision follows the emergence of structural changes in Japanese industry following the rise in the value of the yen and a decline in domestic electric power demand. MITI is planning to work out a revised energy supply/demand outlook through deliberations of committee meetings of the Agency of Natural Resources and Energy, the officials said. They said MITI will also review the breakdown of energy supply sources, including oil, nuclear, coal and natural gas. Nuclear energy provided the bulk of Japan's electric power in the fiscal year ended March 31, supplying an estimated 27 pct on a kilowatt/hour 

In [42]:
len(data)

578

In [4]:
print(*nlp.pipeline, sep='\n')

('tok2vec', <spacy.pipeline.tok2vec.Tok2Vec object at 0x0000021BB79E8D60>)
('tagger', <spacy.pipeline.tagger.Tagger object at 0x0000021BB79E8A60>)
('parser', <spacy.pipeline.dep_parser.DependencyParser object at 0x0000021BB7910740>)
('attribute_ruler', <spacy.pipeline.attributeruler.AttributeRuler object at 0x0000021BB8BE18C0>)
('lemmatizer', <spacy.lang.en.lemmatizer.EnglishLemmatizer object at 0x0000021BB8BEF780>)
('ner', <spacy.pipeline.ner.EntityRecognizer object at 0x0000021BB7910900>)


In [17]:
import pandas as pd
list_ent = []
for i in range(len(doc)):
    for ent in doc[i].ents:
# for ent in doc.ents:
    #check if entity is equal 'LOC' or 'GPE'
        if ent.label_ in ['LOC', 'GPE', 'ORG']:
            if ent.text not in list_ent:
                list_ent.append([ent.text, ent.label_])
# list_ent
df = pd.DataFrame(list_ent, columns=["entity", "type"])
# df.groupby('type').value_counts()
df['type'].value_counts()
df.head(20)

,entity,type
0,JAPAN,GPE
1,The Ministry of International Trade and Industry,ORG
2,MITI,ORG
3,MITI,ORG
4,MITI,ORG
5,the Agency of Natural Resources and Energy,ORG
6,MITI,ORG
7,Japan,GPE
8,ENERGY/U.S. PETROCHEMICAL INDUSTRY Cheap,ORG
9,U.S.,GPE


In [ ]:
# doc = reuters_nlp[51]

# print(*[(e.text, e.label_) for e in doc[i].ents for i in range(len(doc))], sep=' ')

In [13]:
def reset_pipeline(nlp, pipes):
    # remove all custom pipes
    custom_pipes = [pipe for (pipe, _) in nlp.pipeline
                    if pipe not in ['tagger', 'parser', 'ner',
                                    'tok2vec', 'attribute_ruler', 'lemmatizer']]
    for pipe in custom_pipes:
        _ = nlp.remove_pipe(pipe)
    # re-add specified pipes
    for pipe in pipes:
        if 'neuralcoref' == pipe or 'neuralcoref' in str(pipe.__class__):
            nlp.add_pipe(pipe, name='neural_coref')
        else:
            nlp.add_pipe(pipe)

    print(f"Model: {nlp.meta['name']}, Language: {nlp.meta['lang']}")
    print(*nlp.pipeline, sep='\n')
    
reset_pipeline(nlp, ['init_coref'])

ValueError: [E002] Can't find factory for 'init_coref' for language English (en). This usually happens when spaCy calls `nlp.create_pipe` with a custom component name that's not registered on the current language class. If you're using a Transformer, make sure to install 'spacy-transformers'. If you're using a custom component, make sure you've added the decorator `@Language.component` (for function components) or `@Language.factory` (for class components).

Available factories: attribute_ruler, tok2vec, merge_noun_chunks, merge_entities, merge_subtokens, token_splitter, doc_cleaner, parser, beam_parser, entity_linker, ner, beam_ner, entity_ruler, lemmatizer, tagger, morphologizer, senter, sentencizer, textcat, spancat, textcat_multilabel, en.lemmatizer

In [11]:
chunks_list = []
for i in range(1):
    for sent in doc[i].sents:
        for chunk in doc[i].noun_chunks:
            if chunk.text not in chunks_list:
                chunks_list.append([chunk.text, chunk.root.text , chunk.root.dep_,
                chunk.root.head.text])
            # print(chunk.text +'---', chunk.root.text +'---', chunk.root.dep_,
            #     chunk.root.head.text)

In [12]:
chunks_list

[['JAPAN', 'JAPAN', 'nsubj', 'REVISE'],
 ['LONG-TERM ENERGY DEMAND', 'DEMAND', 'dobj', 'REVISE'],
 ['The Ministry', 'Ministry', 'nsubj', 'revise'],
 ['International Trade', 'Trade', 'pobj', 'of'],
 ['Industry', 'Industry', 'conj', 'Trade'],
 ['(MITI', 'MITI', 'appos', 'Ministry'],
 ['its long-term energy supply/demand outlook', 'outlook', 'dobj', 'revise'],
 ['August', 'August', 'pobj', 'by'],
 ['a forecast downtrend', 'downtrend', 'dobj', 'meet'],
 ['Japanese energy demand', 'demand', 'pobj', 'in'],
 ['ministry officials', 'officials', 'nsubj', 'said'],
 ['MITI', 'MITI', 'nsubjpass', 'expected'],
 ['the projection', 'projection', 'dobj', 'lower'],
 ['primary energy supplies', 'supplies', 'pobj', 'for'],
 ['the year', 'year', 'pobj', 'in'],
 ['550 mln kilolitres', 'kilolitres', 'pobj', 'to'],
 ['600 mln', 'mln', 'pobj', 'from'],
 ['they', 'they', 'nsubj', 'said'],
 ['The decision', 'decision', 'nsubj', 'follows'],
 ['the emergence', 'emergence', 'dobj', 'follows'],
 ['structural change

In [ ]:
from spacy import displacy
displacy.render(doc, style='ent')

## Relationship extraction

In [ ]:
from spacy.tokens import Token
from spacy import Language

Token.set_extension('ref_n', default='', force = True)
Token.set_extension('ref_t', default='', force = True)

@Language.component("init_coref")
def init_coref(doc):
    for e in doc.ents:
        if e.label_ in ['ORG', 'GOV', 'PERSON']:
            e[0]._.ref_n, e[0]._.ref_t = e.text, e.label_
    return doc

def reset_pipeline(nlp, pipes):
    # remove all custom pipes
    custom_pipes = [pipe for (pipe, _) in nlp.pipeline
                    if pipe not in ['tagger', 'parser', 'ner',
                                    'tok2vec', 'attribute_ruler', 'lemmatizer']]
    for pipe in custom_pipes:
        _ = nlp.remove_pipe(pipe)
    # re-add specified pipes
    for pipe in pipes:
        if 'neuralcoref' == pipe or 'neuralcoref' in str(pipe.__class__):
            nlp.add_pipe(pipe, name='neural_coref')
        else:
            nlp.add_pipe(pipe)

    print(f"Model: {nlp.meta['name']}, Language: {nlp.meta['lang']}")
    print(*nlp.pipeline, sep='\n')
    
reset_pipeline(nlp, ['init_coref'])

## Using dependency trees

In [ ]:
options = {"bg": "white", "distance": 130,
           "color": "black", "font": "Source Sans Pro"}
for sent in doc.sents:
    displacy.render(sent, style="dep", options=options)
    # displacy.render(doc, style="dep", options=options)

In [ ]:
# Actually we search for the shortest path between the
# subject running through our predicate (verb) to the object.
# subject and object are organizations in our examples.

# Here are the three helper functions omitted in the book:
# - bfs: breadth first searching the closest subject/object 
# - is_passive: checks if noun or verb is in passive form
# - find_subj: searches left part of tree for subject
# - find_obj: searches right part of tree for object

from collections import deque

def bfs(root, ent_type, deps, first_dep_only=False):
    """Return first child of root (included) that matches
    ent_type and dependency list by breadth first search.
    Search stops after first dependency match if first_dep_only
    (used for subject search - do not "jump" over subjects)"""
    to_visit = deque([root]) # queue for bfs

    while len(to_visit) > 0:
        child = to_visit.popleft()
        # print("child", child, child.dep_)
        if child.dep_ in deps:
            if child._.ref_t == ent_type:
                return child
            elif first_dep_only: # first match (subjects)
                return None
        elif child.dep_ == 'compound' and \
             child.head.dep_ in deps and \
             child._.ref_t == ent_type: # check if contained in compound
            return child
        to_visit.extend(list(child.children))
    return None

def is_passive(token):
    if token.dep_.endswith('pass'): # noun
        return True
    for left in token.lefts: # verb
        if left.dep_ == 'auxpass':
            return True
    return False

def find_subj(pred, ent_type, passive):
    """Find closest subject in predicates left subtree or
    predicates parent's left subtree (recursive).
    Has a filter on organizations."""
    for left in pred.lefts:
        if passive: # if pred is passive, search for passive subject
            subj = bfs(left, ent_type, ['nsubjpass', 'nsubj:pass'], True)
        else:
            subj = bfs(left, ent_type, ['nsubj'], True)
        if subj is not None: # found it!
            return subj
    if pred.head != pred and not is_passive(pred): 
        return find_subj(pred.head, ent_type, passive) # climb up left subtree
    else:
        return None

def find_obj(pred, ent_type, excl_prepos):
    """Find closest object in predicates right subtree.
    Skip prepositional objects if the preposition is in exclude list.
    Has a filter on organizations."""
    for right in pred.rights:
        obj = bfs(right, ent_type, ['dobj', 'pobj', 'iobj', 'obj', 'obl'])
        if obj is not None:
            if obj.dep_ == 'pobj' and obj.head.lemma_.lower() in excl_prepos: # check preposition
                continue
            return obj
    return None

def extract_rel_dep(doc, pred_name, pred_synonyms, excl_prepos=[]):
    for token in doc:
        if token.pos_ == 'VERB' and token.lemma_ in pred_synonyms:
            pred = token
            passive = is_passive(pred)
            subj = find_subj(pred, 'ORG', passive)
            if subj is not None:
                obj = find_obj(pred, 'ORG', excl_prepos)
                if obj is not None:
                    if passive: # switch roles
                        obj, subj = subj, obj
                    yield ((subj._.ref_n, subj._.ref_t), pred_name, 
                           (obj._.ref_n, obj._.ref_t))

In [ ]:
# text = """Fujitsu said that Schlumberger Ltd has arranged 
# to buy its stake in Fairchild Inc."""
text = """Iraq said today its troops were pushing Iranian forces out of positions they had initially 
occupied when they launched a new offensive near the southern port of Basra early yesterday."""
doc = nlp(text)
print(*extract_rel_dep(doc, 'push', ['pushing']), sep='\n')

In [ ]:
print(*extract_rel_dep(doc, 'sell', ['sells']), sep='\n')